
# Import Necessary Libraries


# Import libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from natsort import natsorted # type: ignore
from sklearn.model_selection import GroupShuffleSplit
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.models as models
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
import shutil
from sklearn.metrics import precision_score as skl_precision_score
from sklearn.metrics import recall_score as skl_recall_score
from sklearn.metrics import f1_score as skl_f1_score

from custom_dataset import CustomMelanomaDataset  # Import the custom dataset
from resnet_model import ResNetModel


import shutil
#import torch.profiler
#from torch.utils.tensorboard import SummaryWriter
#from tensorboardX import SummaryWriter
#from torch.profiler import profile, ProfilerActivity
#import tkinter as tk
#from tkinter import filedialog, messagebox



<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
# Define the dynamic root path
BASE_DIR = os.path.join(os.path.expanduser('~'), 'Desktop')
root_path = os.path.join(os.path.expanduser('~'), 'Desktop')

# Create the full path to the CSV file
csv_path = os.path.join(BASE_DIR, 'Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth.csv')

# Load the CSV file
df = pd.read_csv(csv_path)

# Display the first few rows of the dataset
print(df.head())

# Display the structure of the dataset
print(df.info())


     image_name  patient_id     sex  age_approx anatom_site_general_challenge  \
0  ISIC_2637011  IP_7279968    male        45.0                     head/neck   
1  ISIC_0015719  IP_3075186  female        45.0               upper extremity   
2  ISIC_0052212  IP_2842074  female        50.0               lower extremity   
3  ISIC_0068279  IP_6890425  female        45.0                     head/neck   
4  ISIC_0074268  IP_8723313  female        55.0               upper extremity   

  diagnosis benign_malignant  target  
0   unknown           benign     0.0  
1   unknown           benign     0.0  
2     nevus           benign     0.0  
3   unknown           benign     0.0  
4   unknown           benign     0.0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33126 entries, 0 to 33125
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   image_name                     33126 non-null 

# Analysing the RAW CSV

In [4]:

# Load the CSV file
df = pd.read_csv(csv_path)

# Ensure 'anatom_site_general_challenge' is treated as a string
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].astype(str)

# Strip extra spaces
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].str.strip()

# Print unique values to identify any anomalies
unique_values = df['anatom_site_general_challenge'].unique()
print("Unique values in 'anatom_site_general_challenge':", unique_values)

# Check for rows that may contain 'unknown' in any form
unknown_variations = df[df['anatom_site_general_challenge'].str.contains('unknown', case=False, na=False, regex=True)]
print("Rows with variations of 'unknown':\n", unknown_variations)

Unique values in 'anatom_site_general_challenge': ['head/neck' 'upper extremity' 'lower extremity' 'torso' 'nan'
 'lower extremityi wrote to him' 'palms/soles' 'oral/genital']
Rows with variations of 'unknown':
 Empty DataFrame
Columns: [image_name, patient_id, sex, age_approx, anatom_site_general_challenge, diagnosis, benign_malignant, target]
Index: []


In [5]:
# Identify rows with potentially problematic entries
problematic_entries = df[df['anatom_site_general_challenge'].str.contains('unknown', case=False, na=False, regex=False)]
print("Problematic entries:\n", problematic_entries)


Problematic entries:
 Empty DataFrame
Columns: [image_name, patient_id, sex, age_approx, anatom_site_general_challenge, diagnosis, benign_malignant, target]
Index: []


In [6]:
# Replace common variations with 'unknown'
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].replace({
    'nan': 'unknown',
    'lower extremityi wrote to him': 'lower extremity',
    # Add other replacements if needed
})

# Check again for 'unknown' values
unknown_values = (df['anatom_site_general_challenge'] == 'unknown').sum()
print("Number of 'unknown' values after cleanup:", unknown_values)

# Verify unique values after cleanup
unique_values = df['anatom_site_general_challenge'].unique()
print("Unique values in 'anatom_site_general_challenge' after cleanup:", unique_values)


Number of 'unknown' values after cleanup: 527
Unique values in 'anatom_site_general_challenge' after cleanup: ['head/neck' 'upper extremity' 'lower extremity' 'torso' 'unknown'
 'palms/soles' 'oral/genital']


In [7]:
print(df[['anatom_site_general_challenge']].head(90))


   anatom_site_general_challenge
0                      head/neck
1                upper extremity
2                lower extremity
3                      head/neck
4                upper extremity
..                           ...
85                     head/neck
86                         torso
87               lower extremity
88               upper extremity
89                         torso

[90 rows x 1 columns]


# checking the anomaly in the original CSV

In [8]:
# Define the path to CSV file
csv_path = os.path.join(os.path.expanduser('~'), 'Desktop', 'Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth.csv')

# Load the CSV file
df = pd.read_csv(csv_path)

# Ensure 'anatom_site_general_challenge' is treated as a string
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].astype(str)

# Strip extra spaces
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].str.strip()

# Replace common variations with 'unknown'
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].replace({
    'nan': 'unknown',
    'lower extremityi wrote to him': 'lower extremity',
    # Add other replacements if needed
})

# Check for NaN values
nan_values = df.isna().sum()
print("NaN values in each column:\n", nan_values)

# Check for infinity values
infinity_values = df.replace([np.inf, -np.inf], np.nan).isna().sum() - df.isna().sum()
print("Infinity values in each column:\n", infinity_values)

# Check for 'unknown' values in 'anatom_site_general_challenge' column
unknown_values = (df['anatom_site_general_challenge'] == 'unknown').sum()
print("Number of 'unknown' values in 'anatom_site_general_challenge':", unknown_values)

# Print unique values to identify variations
unique_values = df['anatom_site_general_challenge'].unique()
print("Unique values in 'anatom_site_general_challenge':", unique_values)

# Check for any variations of 'unknown'
unknown_variations = df[df['anatom_site_general_challenge'].str.contains('unknown', case=False, na=False)]
print("Rows with variations of 'unknown':\n", unknown_variations)

# Check for unusually large values
# Exclude 'diagnosis' column from max value check
columns_for_max = [col for col in df.columns if col != 'diagnosis']
max_values = df[columns_for_max].max(numeric_only=True)  # Ensure numeric_only=True to avoid warnings
print("Maximum values in each column (excluding 'diagnosis'):\n", max_values)


NaN values in each column:
 image_name                        0
patient_id                        0
sex                              65
age_approx                       68
anatom_site_general_challenge     0
diagnosis                         0
benign_malignant                  0
target                            1
dtype: int64
Infinity values in each column:
 image_name                       0
patient_id                       0
sex                              0
age_approx                       0
anatom_site_general_challenge    0
diagnosis                        0
benign_malignant                 0
target                           0
dtype: int64
Number of 'unknown' values in 'anatom_site_general_challenge': 527
Unique values in 'anatom_site_general_challenge': ['head/neck' 'upper extremity' 'lower extremity' 'torso' 'unknown'
 'palms/soles' 'oral/genital']
Rows with variations of 'unknown':
          image_name  patient_id     sex  age_approx  \
33     ISIC_0086462  IP_3200260  female

# Drop the unknown, missing, NaN values from CSV

In [9]:

# Define the base directory and paths
BASE_DIR = os.path.join(os.path.expanduser('~'), 'Desktop')
csv_path = os.path.join(BASE_DIR, 'Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth.csv')
new_csv_path = os.path.join(BASE_DIR, 'Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth_preprocess.csv')

# Load the CSV file
df = pd.read_csv(csv_path)

# Ensure 'anatom_site_general_challenge' is treated as a string and strip extra spaces
df['anatom_site_general_challenge'] = df['anatom_site_general_challenge'].astype(str).str.strip()

# Replace literal 'nan' strings with actual NaN values
df.replace('nan', np.nan, inplace=True)

# Remove rows with 'unknown' in 'anatom_site_general_challenge' column
df = df[df['anatom_site_general_challenge'] != 'unknown']

# Remove rows with NaN or infinity values in any column
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# Drop the 'diagnosis' column
if 'diagnosis' in df.columns:
    df = df.drop(columns=['diagnosis'])

# Check if the new CSV file already exists
if os.path.exists(new_csv_path):
    # Clear the existing file
    open(new_csv_path, 'w').close()

# Save the cleaned DataFrame to the new CSV file
df.to_csv(new_csv_path, index=False)

print(f"Preprocessed CSV saved to {new_csv_path}.")


Preprocessed CSV saved to /Users/hafeez/Desktop/Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth_preprocess.csv.


# Verifying the preprocessing CSV

In [10]:
# Re-load the cleaned CSV
df_cleaned = pd.read_csv(new_csv_path)

# Ensure 'anatom_site_general_challenge' is treated as a string
df_cleaned['anatom_site_general_challenge'] = df_cleaned['anatom_site_general_challenge'].astype(str).str.strip()


# Check for NaN values
nan_values = df_cleaned.isna().sum()
print("NaN values in each column:\n", nan_values)

# Check for infinity values
infinity_values = df_cleaned.replace([np.inf, -np.inf], np.nan).isna().sum() - df.isna().sum()
print("Infinity values in each column:\n", infinity_values)

# Check for 'unknown' values in 'anatom_site_general_challenge' column
unknown_values = (df_cleaned['anatom_site_general_challenge'] == 'unknown').sum()
print("\nNumber of 'unknown' values in 'anatom_site_general_challenge':", unknown_values)

# Print unique values to identify variations
unique_values = df_cleaned['anatom_site_general_challenge'].unique()
print("\nUnique values in 'anatom_site_general_challenge':", unique_values)

# Check for any variations of 'unknown'
unknown_variations = df_cleaned[df_cleaned['anatom_site_general_challenge'].str.contains('unknown', case=False, na=False)]
print("\nRows with variations of 'unknown':\n", unknown_variations)

# Check for unusually large values
# Exclude 'diagnosis' column from max value check
columns_for_max = [col for col in df_cleaned.columns if col != 'diagnosis']
max_values = df_cleaned[columns_for_max].max(numeric_only=True)  # Ensure numeric_only=True to avoid warnings
print("\nMaximum values in each column (excluding 'diagnosis'):\n", max_values)

NaN values in each column:
 image_name                       0
patient_id                       0
sex                              0
age_approx                       0
anatom_site_general_challenge    0
benign_malignant                 0
target                           0
dtype: int64
Infinity values in each column:
 image_name                       0
patient_id                       0
sex                              0
age_approx                       0
anatom_site_general_challenge    0
benign_malignant                 0
target                           0
dtype: int64

Number of 'unknown' values in 'anatom_site_general_challenge': 0

Unique values in 'anatom_site_general_challenge': ['head/neck' 'upper extremity' 'lower extremity' 'torso' 'palms/soles'
 'oral/genital']

Rows with variations of 'unknown':
 Empty DataFrame
Columns: [image_name, patient_id, sex, age_approx, anatom_site_general_challenge, benign_malignant, target]
Index: []

Maximum values in each column (excluding 'diag

# Group by split of NEW CSV - Split by Patient ID

In [11]:
# Step 1: Define the universal path handling logic
BASE_DIR = os.path.join(os.path.expanduser('~'), 'Desktop')
SPLIT_CSV_DIR = os.path.join(BASE_DIR, 'Thesis_Hafeez', 'Dataset', 'split_csv')

# Universal Path Setup for Images
IMAGE_DIR = os.path.join(BASE_DIR, 'Thesis_Hafeez', 'Dataset', 'Train_JPEG', 'JPEG')

# Paths for Train/Test CSVs
TRAIN_CSV_PATH = os.path.join(BASE_DIR, 'Thesis_Hafeez', 'Dataset', 'split_csv', 'train_split.csv')
TEST_CSV_PATH = os.path.join(BASE_DIR, 'Thesis_Hafeez', 'Dataset', 'split_csv', 'test_split.csv')

# Create the full path to the CSV file
CSV_PATH = os.path.join(root_path, 'Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth.csv')
new_csv_path = os.path.join(BASE_DIR, 'Thesis_Hafeez/Dataset/Train_JPEG/ISIC_2020_Training_GroundTruth_preprocess.csv')


MODEL_PATH = os.path.join(BASE_DIR, 'Thesis_Hafeez',  'Thesis_Code/Enhanced-Skin-Lesion-detection-using-Deep-Learning-model/results', 'output')

In [12]:
# Ensure that the directory exists
if not os.path.exists(SPLIT_CSV_DIR):
    os.makedirs(SPLIT_CSV_DIR)

# Define paths for train and test CSV files
train_csv_path = os.path.join(SPLIT_CSV_DIR, 'train_split.csv')
test_csv_path = os.path.join(SPLIT_CSV_DIR, 'test_split.csv')

# Check if train_split.csv and test_split.csv exist; if not, create them
if not os.path.exists(train_csv_path):
    open(train_csv_path, 'w').close()  # Create the file if it doesn't exist

if not os.path.exists(test_csv_path):
    open(test_csv_path, 'w').close()  # Create the file if it doesn't exist

# Step 2: Split the dataset by `patient_id`, ensuring no patient data is split across train and test sets
# Group by patient_id
grouped = df_cleaned.groupby('patient_id')

# Extract unique patient_ids
unique_patient_ids = df_cleaned['patient_id'].unique()

# Split the unique patient ids into 75% train and 25% test
train_patient_ids, test_patient_ids = train_test_split(unique_patient_ids, test_size=0.25, random_state=42)

# Filter the dataframe to create train and test datasets
train_df = df_cleaned[df_cleaned['patient_id'].isin(train_patient_ids)]
test_df = df_cleaned[df_cleaned['patient_id'].isin(test_patient_ids )]

# Display the size of the splits
print(f"Training set size: {train_df.shape}")
print(f"Test set size: {test_df.shape}")

# Step 3: Function to check, clear, and write CSV
def write_csv(dataframe, path):
    if os.path.exists(path):
        # If CSV exists, remove its contents
        with open(path, 'w') as f:
            pass  # Clearing the file
    # Write the new data to the CSV
    dataframe.to_csv(path, index=False)

# Step 4: Write to train and test CSV
write_csv(train_df, train_csv_path)
write_csv(test_df, test_csv_path)

print(f"Train and test CSV files have been successfully written to {SPLIT_CSV_DIR}.")


Training set size: (24300, 7)
Test set size: (8230, 7)
Train and test CSV files have been successfully written to /Users/hafeez/Desktop/Thesis_Hafeez/Dataset/split_csv.


In [13]:
# Print the number of images in the train and test datasets
print(f"Total number of images in the train set: {len(train_df['image_name'])}")
print(f"Total number of images in the test set: {len(test_df['image_name'])}")

# Optionally, you can also check the number of unique patient IDs in each split:
print(f"Total unique patients in the train set: {train_df['patient_id'].nunique()}")
print(f"Total unique patients in the test set: {test_df['patient_id'].nunique()}")


Total number of images in the train set: 24300
Total number of images in the test set: 8230
Total unique patients in the train set: 1538
Total unique patients in the test set: 513


In [15]:
class CustomResNetModel(nn.Module):
    def __init__(self, num_metadata_features):
        super(CustomResNetModel, self).__init__()
        
        # Load a pre-trained ResNet-18 model
        self.resnet = models.resnet18(pretrained=True)
        
        # Modify the fully connected layer to accommodate the output of ResNet
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # Remove the original classification layer
        
        # Define additional layers for metadata processing
        self.metadata_fc = nn.Sequential(
            nn.Linear(num_metadata_features, 64),  # Example hidden layer
            nn.ReLU(),
            nn.Linear(64, 32),  # Another hidden layer
            nn.ReLU()
        )
        
        # Combine features from ResNet and metadata
        self.combined_fc = nn.Sequential(
            nn.Linear(in_features + 32, 64),  # Combine ResNet features and metadata features
            nn.ReLU(),
            nn.Linear(64, 1)  # Output layer for binary classification
        )
        
    def forward(self, images, metadata):
        # Pass images through the ResNet model
        resnet_features = self.resnet(images)
        
        # Process metadata
        metadata_features = self.metadata_fc(metadata)
        
        # Concatenate ResNet features and metadata features
        combined_features = torch.cat((resnet_features, metadata_features), dim=1)
        
        # Pass through the final fully connected layers
        output = self.combined_fc(combined_features)
        
        return output


# Define Transformations

In [16]:

# Define transforms for image augmentation and normalization
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size (224x224)
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally for data augmentation
    transforms.RandomVerticalFlip(),    # Randomly flip the image vertically
    transforms.ToTensor(),  # Convert PIL Image or numpy.ndarray to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images with pre-defined mean and std
])

# Define transformations for testing
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size (224x224)
    transforms.ToTensor(),  # Convert PIL Image or numpy.ndarray to Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images with pre-defined mean and std
])


# Define train and test datasets


In [17]:
# Initialize datasets
# Create datasets
train_dataset = CustomMelanomaDataset(
    csv_file=train_csv_path,
    image_dir=IMAGE_DIR,
    transform=train_transforms,
    is_test=False  # Indicates that this dataset is for training


)

test_dataset = CustomMelanomaDataset(
    csv_file=test_csv_path,
    image_dir=IMAGE_DIR,
    transform=test_transforms,
    is_test=True  # Indicates that this dataset is for testing
)

# configs

In [18]:
# Learning parameters
lr = 1e-4
NUM_EPOCHS = 10
BATCH_SIZE = 16
num_workers=0

# Determine if CUDA is available
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PIN_MEMORY = True if torch.cuda.is_available() else False

print(f"[INFO] Using device: {DEVICE}")


[INFO] Using device: cpu


# Define dataloaders, lossFunc, Optim

In [19]:
# Create DataLoader instances
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,  # Shuffle the data for training
    num_workers=num_workers,  # Number of workers for data loading
    pin_memory=PIN_MEMORY  # Use pin memory if using CUDA
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # No need to shuffle test data
    num_workers=num_workers,  # Number of workers for data loading
    pin_memory=PIN_MEMORY  # Use pin memory if using CUDA
)

In [20]:
# Calculate steps per epoch
trainSteps = len(train_loader)
testSteps = len(test_loader)

print(f"[INFO] Training steps per epoch: {trainSteps}")
print(f"[INFO] Testing steps per epoch: {testSteps}")


[INFO] Training steps per epoch: 1519
[INFO] Testing steps per epoch: 515


In [21]:
#df_cleaned = pd.read_csv(new_csv_path)
#metadata = df_cleaned.to_dict(orient="records")

#targets = df_cleaned['target'].tolist()  # Replace 'target_column_name' with actual column name


In [22]:
# Assuming 'train_dataset' is an instance of 'CustomMelanomaDataset'
#train_dataset = CustomMelanomaDataset(IMAGE_DIR, metadata, targets)

# Test preprocessing function separately
#sample_metadata = {'sex': 'male', 'age_approx': 25, 'anatom_site_general_challenge': 'upper extremity'}
#processed_metadata = train_dataset.preprocess_metadata(sample_metadata)
#print(f"Processed metadata: {processed_metadata}")


 # Initialize Model, Loss Function, and Optimizer

In [23]:
# Define the model, where we pass the number of metadata features (3 in this case)
num_metadata_features = 3  # Number of metadata features: sex, age, and site
model = CustomResNetModel(num_metadata_features).to(DEVICE)

# Loss function and optimizer
lossFunc = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


/Users/hafeez/opt/anaconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/hafeez/opt/anaconda3/envs/env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
print(model)

CustomResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

# Initialize Training History Dictionary

In [25]:
# Dictionary to store training history
H = {
    "train_loss": [], 
    "train_acc": [], 
    "train_precision": [], 
    "train_recall": [], 
    "train_f1": [], 
    "train_roc_auc": [],
    "test_roc_auc": [],  # Only include ROC AUC for test set
    "test_precision": [],"test_recall": [],"test_average_precision": [],
}


#  Training Loop Implementation with Metric Tracking and Validation

In [26]:

# Define metric calculation functions
def accuracy_score(preds, labels):
    preds = torch.sigmoid(preds).round()  # No need for conversion, assume preds is already a tensor
    correct = (preds.squeeze() == labels).sum().item()
    return correct / labels.size(0)

def precision_score(preds, labels):
    preds = torch.sigmoid(preds).round()  # Ensure preds is tensor
    return skl_precision_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), zero_division=0)

def recall_score(preds, labels):
    preds = torch.sigmoid(preds).round()  # Ensure preds is tensor
    return skl_recall_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), zero_division=0)

def f1_score(preds, labels):
    preds = torch.sigmoid(preds).round()  # Ensure preds is tensor
    return skl_f1_score(labels.cpu().detach().numpy(), preds.cpu().detach().numpy(), zero_division=0)

In [27]:

if __name__ == "__main__":
    startTime = time.time()
    
    for epoch in range(NUM_EPOCHS):
        # Print epoch information
        print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS} running...")
        
        model.train()
        running_train_loss = 0.0
        all_train_preds = []
        all_train_targets = []

        # Training
        with tqdm(total=len(train_loader), desc='Training', unit='batch') as pbar:
            for images, metadata, targets in train_loader:
                images, metadata, targets = images.to(DEVICE), metadata.to(DEVICE), targets.to(DEVICE)
                
                optimizer.zero_grad()
                outputs = model(images, metadata)
                loss = lossFunc(outputs.squeeze(), targets)
                loss.backward()
                optimizer.step()
                
                running_train_loss += loss.item() * images.size(0)
                
                # Collect predictions and targets
                all_train_preds.extend(outputs.detach())  # Keep as tensors
                all_train_targets.extend(targets.detach()) # Keep as tensors
                
                pbar.update(1)  # Update progress bar
        
        avg_train_loss = running_train_loss / len(train_loader.dataset)
        
        # Convert collected tensors to a stacked tensor (needed for metric functions)
        all_train_preds = torch.stack(all_train_preds)  # Convert list of tensors to tensor
        all_train_targets = torch.stack(all_train_targets)  # Convert list of tensors to tensor
        
        # Metrics calculation for training
        train_acc = accuracy_score(all_train_preds, all_train_targets)
        train_precision = precision_score(all_train_preds, all_train_targets)
        train_recall = recall_score(all_train_preds, all_train_targets)
        train_f1 = f1_score(all_train_preds, all_train_targets)
        train_roc_auc = roc_auc_score(all_train_targets.cpu().detach().numpy(), torch.sigmoid(all_train_preds).cpu().detach().numpy())  # Compute ROC AUC

        model.eval()
        all_test_preds = []
        all_test_probs = []

        with torch.no_grad():
            # Testing (without Y targets)
            with tqdm(total=len(test_loader), desc='Testing', unit='batch') as pbar:
                for images, metadata, _ in test_loader:
                    images, metadata = images.to(DEVICE), metadata.to(DEVICE)
                    
                    outputs = model(images, metadata)
                    preds = torch.sigmoid(outputs).squeeze().cpu().detach().numpy()
                    
                    # Collect predictions (no targets available)
                    all_test_preds.extend(preds)
                    all_test_probs.extend(preds)
                    
                    pbar.update(1)  # Update progress bar

        # Metrics calculation for test set (no targets)
        # Compute ROC AUC and Precision-Recall without true Y labels (unsupervised)
        test_roc_auc = roc_auc_score(np.zeros(len(all_test_probs)), all_test_probs)  # Assuming no ground truth, using zeros
        
        # Compute Precision-Recall Curve and Average Precision Score (without true labels)
        precision, recall, _ = precision_recall_curve(np.zeros(len(all_test_probs)), all_test_probs)
        average_precision = average_precision_score(np.zeros(len(all_test_probs)), all_test_probs)

        # Store metrics in history
        H["train_loss"].append(avg_train_loss)
        H["train_acc"].append(train_acc)
        H["train_precision"].append(train_precision)
        H["train_recall"].append(train_recall)
        H["train_f1"].append(train_f1)
        H["train_roc_auc"].append(train_roc_auc)
        H["test_roc_auc"].append(test_roc_auc)
        H["test_precision"].append(precision)
        H["test_recall"].append(recall)
        H["test_average_precision"].append(average_precision)
        
        # Print the results for this epoch
        print(f"[INFO] EPOCH: {epoch + 1}/{NUM_EPOCHS}")
        print(f"Train loss: {avg_train_loss:.6f}")
        print(f"Train Accuracy: {train_acc:.6f}")
        print(f"Train Precision: {train_precision:.6f}")
        print(f"Train Recall: {train_recall:.6f}")
        print(f"Train F1 Score: {train_f1:.6f}")
        print(f"Train ROC AUC: {train_roc_auc:.6f}")
        print(f"Test ROC AUC: {test_roc_auc:.6f}")
        print(f"Test Average Precision Score: {average_precision:.6f}")
        print()

    endTime = time.time()
    print(f"[INFO] Total time taken to train the model: {np.round(endTime - startTime)} seconds")


Training:   0%|          | 0/1519 [00:00<?, ?batch/s]


Epoch 1/10 running...


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/hafeez/opt/anaconda3/envs/env/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/hafeez/opt/anaconda3/envs/env/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'CustomMelanomaDataset' on <module '__main__' (built-in)>
Training:   0%|          | 0/1519 [01:02<?, ?batch/s]


KeyboardInterrupt: 

# Plot the Training Metrics 

In [ ]:

def plot_metrics(H):
    epochs = range(1, len(H["train_loss"]) + 1)

    plt.figure(figsize=(16, 12))

    # Plot Training Loss
    plt.subplot(3, 3, 1)
    plt.plot(epochs, H["train_loss"], 'b', label='Train Loss')
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(3, 3, 2)
    plt.plot(epochs, H["train_acc"], 'b', label='Train Accuracy')
    plt.title('Training Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Precision
    plt.subplot(3, 3, 3)
    plt.plot(epochs, H["train_precision"], 'b', label='Train Precision')
    plt.title('Training Precision')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.legend()

    # Plot Recall
    plt.subplot(3, 3, 4)
    plt.plot(epochs, H["train_recall"], 'b', label='Train Recall')
    plt.title('Training Recall')
    plt.xlabel('Epochs')
    plt.ylabel('Recall')
    plt.legend()

    # Plot F1 Score
    plt.subplot(3, 3, 5)
    plt.plot(epochs, H["train_f1"], 'b', label='Train F1 Score')
    plt.title('Training F1 Score')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.legend()

    # Plot ROC AUC Score
    plt.subplot(3, 3, 6)
    plt.plot(epochs, H["train_roc_auc"], 'b', label='Train ROC AUC')
    plt.plot(epochs, H["test_roc_auc"], 'r', label='Test ROC AUC')
    plt.title('ROC AUC Score')
    plt.xlabel('Epochs')
    plt.ylabel('ROC AUC')
    plt.legend()

    # Plot Precision-Recall Curve
    plt.subplot(3, 3, 7)
    for i, (precision, recall) in enumerate(zip(H["test_precision"], H["test_recall"])):
        plt.plot(recall, precision, label=f'Epoch {i+1}')
    plt.title('Precision-Recall Curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend()

    # Plot Average Precision Score
    plt.subplot(3, 3, 8)
    plt.plot(epochs, H["test_average_precision"], 'b', label='Test Average Precision')
    plt.title('Average Precision Score')
    plt.xlabel('Epochs')
    plt.ylabel('Average Precision')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Plot the metrics
plot_metrics(H)

In [ ]:

# Plot ROC AUC for testing
plt.subplot(2, 2, 2)
plt.plot(H["test_roc_auc"], label='Test ROC AUC', color='orange')
plt.xlabel('Epoch')
plt.ylabel('ROC AUC')
plt.title('Testing ROC AUC')
plt.legend()
